# Semantic search quick start

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/search/00-quick-start.ipynb)

This interactive notebook will introduce you to some basic operations with Elasticsearch, using the official [Elasticsearch Python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html).
You'll perform semantic search using [Sentence Transformers](https://www.sbert.net) for text embedding. Learn how to integrate traditional text-based search with semantic search, for a hybrid search system.

## Create Elastic Cloud deployment

If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?onboarding_token=vectorsearch&utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial.

Once logged in to your Elastic Cloud account, go to the [Create deployment](https://cloud.elastic.co/deployments/create) page.
1. Select **Create deployment**.
2. Then choose the **Elasticsearch** option & click next.
3. Write **tmls-2025-workshop** as name for your elasticsearch deployment.
4. Select **AWS** as Cloud Provider.
5. Select **🇺🇸   N. Virginia (us-east-1)** as Region
6. Select **Vector Search Optimized (ARM)** as Hardware Profile
7. Leave all other settings including **Version** with their default values.
8. Then click **Create hosted deployment**. This will take about 5 minutes.
9. You'll be provided with a username and password you can ignore that; as we will not need that for this workshop.
10. Once the deployment is ready hit **Continue**

Your setup page should look like this (Insert Image TBD)

## Install packages and import modules

To get started, we'll need to connect to our Elastic deployment using the Python client.
Because we're using an Elastic Cloud deployment, we'll use the **Cloud ID** to identify our deployment.

First we need to install the `elasticsearch` Python client.

In [1]:
!pip install -qU elasticsearch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.3/914.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 2.5 MB/s eta 0:00:00


## Initialize the Elasticsearch client

Now we can instantiate the [Elasticsearch python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/index.html), providing the cloud id and password in your deployment.

In [2]:
from elasticsearch import Elasticsearch, helpers, exceptions
from urllib.request import urlopen
from getpass import getpass
import json
import time

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

# Create the client instance
client = Elasticsearch(
    ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
)

Elastic Cloud ID: ··········
Elastic Api Key: ··········


In [3]:
# check if connection is succesfully established
if client.ping():
    print("Connected to Elasticsearch! 🥳")
    print(client.info())
else:
    print("Connection failed.")

Connected to Elasticsearch! 🥳
{'name': 'instance-0000000000', 'cluster_name': 'de9c18bb54f84824b798e9ae1fb0e88d', 'cluster_uuid': 'yz5MrG9STE6vDTE_ErIA4Q', 'version': {'number': '9.0.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '0a58bc1dc7a4ae5412db66624aab968370bd44ce', 'build_date': '2025-05-28T10:06:37.834829258Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'}


## Platform Walkthrough

In [4]:
model_id = ".multilingual-e5-small-elasticsearch"
client.ingest.put_pipeline(
    id="title_embeddings_pipeline",
    description="Ingest pipeline for converting title to embeddings using elastic multilingual e5-small model",
    processors=[
        {
            "inference": {
                "model_id": model_id,
                "input_output": [
                    {"input_field": "title", "output_field": "title_embedding"}
                ],
            }
        }
    ],
)

ObjectApiResponse({'acknowledged': True})

Let's note a few important parameters from that API call:

- `inference`: A processor that performs inference using a machine learning model.
- `model_id`: Specifies the ID of the machine learning model to be used. In this example, the model ID is set to `.multilingual-e5-small-elasticsearch`.
- `input_output`: Specifies input and output fields
- `input_field`: Field name from which the `dense_vector` representation are created.
- `output_field`:  Field name which contains inference results.

## Define the mapping
Now we need to create an index **tmls-optimized-rag-workshop**

Notice how we configured the mappings specifically:

1. Setting the **index_options.type** field of the **title_embedding** to **bbq_hnsw** indicating it should be a binary vector. Important to note that the full precision embedding is not lost, and we can use the non-quantized, original vectors to re-rank the candidates to get the top results. This combines:

  - The performance and memory gains of approximate retrieval using quantized vectors for retrieving the top candidates.
  - The accuracy of using the original vectors for rescoring the top candidates.

2. We defined plot_semantic as a semantic_text field. We have configured the plot field to copy its value to the plot_semantic field. While copy_to is not required to use semantic_text, it enables use cases like hybrid search where semantic and lexical techniques are used together.


In [6]:
index_name = "tmls-optimized-rag-workshop"
client.indices.delete(index=index_name, ignore_unavailable=True)
client.indices.create(
    index=index_name,
    settings={"index": {"default_pipeline": "title_embeddings_pipeline"}},
    mappings={
        "properties": {
            "title": {
                "type": "text"
            },
            "title_embedding": {
                "type": "dense_vector",
                "dims": 384,
                "index": "true",
                "similarity": "cosine",
                "index_options": {
                    "type": "bbq_hnsw"
                }
            },
            "genre": {
                "type": "text"
            },
            "plot": {
                "type": "text",
                "copy_to": "plot_semantic"
            },
            "plot_semantic": {
                "type": "semantic_text"
            },
            "released": {
                "type": "integer"
            },
        }
    },
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'tmls-optimized-rag-workshop'})

### Index test data

Run the following command to upload a test data, containing information about 12 popular movies books from this [dataset](https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/data.json)

In [8]:
#url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/movies.json"
#response = urlopen(url)

# Load the response data into a JSON object
with open('/content/movies_with_wikipedia_plots.json', 'r') as f:
    data_json = json.load(f)

# Prepare the documents to be indexed
documents = []
for doc in data_json:
    documents.append(
        {
            "_index": index_name,
            "_source": doc,
        }
    )

In [9]:
%%time
# Use helpers.bulk to index
helpers.bulk(client, documents, request_timeout=600)

print(f"Done indexing documents into {index_name} index!")
time.sleep(3)

<timed exec>:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.


Done indexing documents into tmls-optimized-rag-workshop index!
CPU times: user 210 ms, sys: 31.9 ms, total: 242 ms
Wall time: 41.9 s


## Semantic Search
Now that our index is populated, we can query it using semantic search.

Aside: Pretty printing Elasticsearch search results
Your search API calls will return hard-to-read nested JSON. We'll create a little function called pretty_search_response to return nice, human-readable outputs from our examples.

In [10]:
def pretty_search_response(response, highlight = False):
    if len(response["hits"]["hits"]) == 0:
        print("Your search returned no results.")
    else:
        for hit in response["hits"]["hits"]:
            id = hit["_id"]
            score = hit["_score"]
            title = hit["_source"]["title"]
            runtime = hit["_source"]["runtime"]
            plot = hit["_source"]["plot"]
            keyScene = hit["_source"]["keyScene"]
            genre = hit["_source"]["genre"]
            released = hit["_source"]["released"]

            pretty_output = f"\nID: {id}\nScore: {score}\nTitle: {title}\nRuntime: {runtime}\nPlot: {plot}\nKey Scene: {keyScene}\nGenre: {genre}\nReleased: {released}"
            if highlight:
              highlight_plot_semantic = hit["highlight"]['plot_semantic']
              for section_number, section in enumerate(highlight_plot_semantic):
                pretty_output += f"\nHighlighted Section {section_number}: {section}"

            print(pretty_output)

### Semantic Search with the `semantic` Query

We can use the [semantic query](https://www.elastic.co/guide/en/elasticsearch/reference/master/query-dsl-semantic-query.html) to quickly & easily query the `semantic_text` field in our index.
Under the hood, an embedding is automatically generated for our query text using the `semantic_text` field's inference endpoint.

In [11]:
query_text = "organized crime movies"

In [12]:
response = client.search(
    index=index_name,
    query={
        "semantic": {
            "field": "plot_semantic",
            "query": query_text
        }
    },
    size=3
)

pretty_search_response(response)


ID: 9UmNe5cBCq_DruP2MK46
Score: 12.085434
Title: Goodfellas
Runtime: 146
Plot: In 1955, teenager Henry Hill becomes enamored by the criminal life and Mafia presence in East New York, a working-class Italian-American neighborhood in Brooklyn, New York City. He begins working for local caporegime Paulie Cicero and his associates Jimmy Conway, an Irish-American truck hijacker and gangster, and Tommy DeVito, a fellow juvenile delinquent. Henry begins as a fence for Jimmy, gradually working his way up to more serious crimes.Throughout the 1960s, the three men excel at carjacking, stealing cargo trucks from JFK Airport, and eventually commit the Air France Robbery. They spend most of their nights at the Copacabana nightclub, carousing with women. Henry starts dating Karen Friedman, a Jewish woman who is initially confused by Henry's criminal activities. She is soon seduced by Henry's glamorous lifestyle, and marries him, despite her parents' disapproval.In 1970, Billy Batts, a made man in t

## Semantic Highlighting

In [13]:
response = client.search(
    index=index_name,
    query={
        "semantic": {
            "field": "plot_semantic",
            "query": query_text
        }
    },
    highlight={
      "fields": {
        "plot_semantic": {"number_of_fragments" : 2, "order" : "score"}
      }
    },
    size=3
)

pretty_search_response(response, highlight=True)


ID: 9UmNe5cBCq_DruP2MK46
Score: 12.085434
Title: Goodfellas
Runtime: 146
Plot: In 1955, teenager Henry Hill becomes enamored by the criminal life and Mafia presence in East New York, a working-class Italian-American neighborhood in Brooklyn, New York City. He begins working for local caporegime Paulie Cicero and his associates Jimmy Conway, an Irish-American truck hijacker and gangster, and Tommy DeVito, a fellow juvenile delinquent. Henry begins as a fence for Jimmy, gradually working his way up to more serious crimes.Throughout the 1960s, the three men excel at carjacking, stealing cargo trucks from JFK Airport, and eventually commit the Air France Robbery. They spend most of their nights at the Copacabana nightclub, carousing with women. Henry starts dating Karen Friedman, a Jewish woman who is initially confused by Henry's criminal activities. She is soon seduced by Henry's glamorous lifestyle, and marries him, despite her parents' disapproval.In 1970, Billy Batts, a made man in t

### Dense vector Search

This example will:

Search using approximate kNN for the top **12** candidates.
Rescore the top **9** candidates (oversample * k) per shard using the original, non quantized vectors.
Return the top **3** (k) rescored candidates.
Merge the rescored canddidates from all shards, and return the top **3** (k) results.

In [14]:
response = client.search(
    index=index_name,
    knn={
        "field": "title_embedding",
        "query_vector_builder": {
          "text_embedding": {
            "model_id": model_id,
            "model_text": query_text
          }
        },
        "rescore_vector": {
            "oversample": 3
        },
        "k": 3,
        "num_candidates": 12
    }
)

pretty_search_response(response)


ID: 8UmNe5cBCq_DruP2MK46
Score: 0.90041786
Title: Fight Club
Runtime: 139
Plot: The unnamed Narrator, who struggles with insomnia and dissatisfaction with his job and lifestyle, finds temporary solace in support groups. As his insomnia worsens, he discovers that expressions of emotional vulnerability help him sleep, leading him to join multiple groups for people facing emotionally distressing problems, despite his expressions being fraudulent. His efforts are thwarted when Marla Singer, another impostor, joins the same groups. The Narrator cannot present his fabricated struggles as genuine, or divert his attention from her presence as an impostor, causing his sleeplessness to return. He arranges for them to attend different sessions to regain his ability to sleep and, under certain circumstances, to exchange contact information, to which she reluctantly agrees.On a return flight from work, the Narrator meets a soap salesman, Tyler Durden. After an explosion destroys the Narrator's apa

## Full - text Search
#### Match query
Returns documents that `match` a provided text, number, date or boolean value. The provided text is analyzed before matching.

The `match` query is the standard query for performing a full-text search, including options for fuzzy matching.

[Read more](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query.html#match-query-ex-request).

In [10]:
response = client.search(
    index=index_name,
    query = {
        "match": {
            "plot": {
                "query": query_text
            }
        }
    },
    size=3
)

pretty_search_response(response)


ID: zkkke5cBCq_DruP2sq4t
Score: 5.1212735
Title: The Dark Knight
Runtime: 152
Plot: The Dark Knight is a 2008 superhero film directed by Christopher Nolan, from a screenplay co-written with his brother Jonathan. Based on the DC Comics superhero Batman, it is the sequel to Batman Begins (2005), and the second installment in The Dark Knight trilogy. The plot follows the vigilante Batman, police lieutenant James Gordon, and district attorney Harvey Dent, who form an alliance to dismantle organized crime in Gotham City. Their efforts are derailed by the Joker, an anarchistic mastermind who seeks to test how far Batman will go to save the city from chaos. The ensemble cast includes Christian Bale, Michael Caine, Heath Ledger, Gary Oldman, Aaron Eckhart, Maggie Gyllenhaal, and Morgan Freeman.Warner Bros. Pictures prioritized a sequel following the successful reinvention of the Batman film series with Batman Begins. Christopher and Batman Begins co-writer David S. Goyer developed the story e

#### Multi-match query

The `multi_match` query builds on the match query to allow multi-field queries.

[Read more](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-multi-match-query.html).

In [11]:
response = client.search(
   index=index_name,
    query={
        "multi_match": {
            "query": query_text,
            "fields": ["plot", "title"]
        }
    },
    size=3
)
pretty_search_response(response)


ID: zkkke5cBCq_DruP2sq4t
Score: 5.1212735
Title: The Dark Knight
Runtime: 152
Plot: The Dark Knight is a 2008 superhero film directed by Christopher Nolan, from a screenplay co-written with his brother Jonathan. Based on the DC Comics superhero Batman, it is the sequel to Batman Begins (2005), and the second installment in The Dark Knight trilogy. The plot follows the vigilante Batman, police lieutenant James Gordon, and district attorney Harvey Dent, who form an alliance to dismantle organized crime in Gotham City. Their efforts are derailed by the Joker, an anarchistic mastermind who seeks to test how far Batman will go to save the city from chaos. The ensemble cast includes Christian Bale, Michael Caine, Heath Ledger, Gary Oldman, Aaron Eckhart, Maggie Gyllenhaal, and Morgan Freeman.Warner Bros. Pictures prioritized a sequel following the successful reinvention of the Batman film series with Batman Begins. Christopher and Batman Begins co-writer David S. Goyer developed the story e

Individual fields can be boosted with the caret (^) notation. Note in the following query how the score of the results that have "JavaScript" in their title is multiplied.

In [12]:
response = client.search(
   index=index_name,
    query={
        "multi_match": {
            "query": query_text,
            "fields": ["plot^3", "title"]
        }
    },
    size=3
)
pretty_search_response(response)


ID: zkkke5cBCq_DruP2sq4t
Score: 15.363821
Title: The Dark Knight
Runtime: 152
Plot: The Dark Knight is a 2008 superhero film directed by Christopher Nolan, from a screenplay co-written with his brother Jonathan. Based on the DC Comics superhero Batman, it is the sequel to Batman Begins (2005), and the second installment in The Dark Knight trilogy. The plot follows the vigilante Batman, police lieutenant James Gordon, and district attorney Harvey Dent, who form an alliance to dismantle organized crime in Gotham City. Their efforts are derailed by the Joker, an anarchistic mastermind who seeks to test how far Batman will go to save the city from chaos. The ensemble cast includes Christian Bale, Michael Caine, Heath Ledger, Gary Oldman, Aaron Eckhart, Maggie Gyllenhaal, and Morgan Freeman.Warner Bros. Pictures prioritized a sequel following the successful reinvention of the Batman film series with Batman Begins. Christopher and Batman Begins co-writer David S. Goyer developed the story e

### Using Filters with Queries
Filters are often added to search queries with the intention of limiting the search to a subset of the documents. A filter can cleanly eliminate documents from a search, without altering the relevance scores of the results.

The next example returns movies that were released after 1990.

In [13]:
response = client.search(
   index=index_name,
   query={
        "bool": {
            "must": [{"match": {"plot": {"query": query_text}}}],
            "must_not": [{"range": {"released": {"lte": 1991}}}],
        }
    },
    size=3
)
pretty_search_response(response)


ID: zkkke5cBCq_DruP2sq4t
Score: 5.1212735
Title: The Dark Knight
Runtime: 152
Plot: The Dark Knight is a 2008 superhero film directed by Christopher Nolan, from a screenplay co-written with his brother Jonathan. Based on the DC Comics superhero Batman, it is the sequel to Batman Begins (2005), and the second installment in The Dark Knight trilogy. The plot follows the vigilante Batman, police lieutenant James Gordon, and district attorney Harvey Dent, who form an alliance to dismantle organized crime in Gotham City. Their efforts are derailed by the Joker, an anarchistic mastermind who seeks to test how far Batman will go to save the city from chaos. The ensemble cast includes Christian Bale, Michael Caine, Heath Ledger, Gary Oldman, Aaron Eckhart, Maggie Gyllenhaal, and Morgan Freeman.Warner Bros. Pictures prioritized a sequel following the successful reinvention of the Batman film series with Batman Begins. Christopher and Batman Begins co-writer David S. Goyer developed the story e

# Querying Documents with Hybrid Search

Now we need to perform a query using two different search strategies:
- Semantic search using the **plot_semantic** field
- Keyword search using the **plot** field

We then use [Reciprocal Rank Fusion (RRF)](https://www.elastic.co/guide/en/elasticsearch/reference/current/rrf.html) to balance the scores to provide a final list of documents, ranked in order of relevance. RRF is a ranking algorithm for combining results from different information retrieval strategies.

Note: With the retriever API, _score contains the document’s relevance score, and the rank is simply the position in the results (first result is rank 1, etc.).

In [14]:
response = client.search(
    index=index_name,
    size=5,
    retriever={
        "rrf": {
            "retrievers": [
                {
                    "standard": {
                      "query": {
                        "match": {
                            "plot": {"query": query_text},
                        }
                    }
                  }
                },
                {
                    "standard": {
                      "query": {
                        "match": {
                            "plot_semantic": {"query": query_text},
                        }
                    }
                  }
                }
            ]
        }
    },
    request_timeout=90
)
pretty_search_response(response)

<ipython-input-14-1781756499>:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  response = client.search(



ID: 1kkke5cBCq_DruP2sq4t
Score: 0.032002047
Title: The Godfather
Runtime: 175
Plot: In 1945, the don of New York City's Corleone family, Vito Corleone, listens to requests during his daughter Connie's wedding to Carlo Rizzi. Vito's youngest son Michael, a Marine and World War II hero who has so far stayed out of the family business, introduces his girlfriend Kay Adams to his family at the reception. Johnny Fontane, a popular singer and Vito's godson, seeks Vito's help in securing a movie role. Vito sends his consigliere, Tom Hagen, to persuade studio president Jack Woltz to offer Johnny the part. Woltz refuses Hagen's request at first, but soon complies after finding the severed head of his prized stud horse in his bed.As Christmas approaches, drug baron Virgil The Turk Sollozzo asks Vito to invest in his narcotics business to provide police protection. Vito declines, citing that involvement in narcotics would alienate his political connections. Suspicious of Sollozzo's partnership wi

In [15]:
response = client.search(
    index=index_name,
    size=5,
    retriever={
        "rrf": {
            "retrievers": [
                {
                    "standard": {
                      "query": {
                        "bool": {
                            "must": [{"match": {"plot": {"query": query_text}}}],
                            "must_not": [{"range": {"released": {"lte": 1991}}}],
                        }
                    }
                  }
                },
                {
                    "standard": {
                      "query": {
                        "match": {
                            "plot_semantic": {"query": query_text},
                        }
                    }
                  }
                }
            ]
        }
    },
    request_timeout=90
)
pretty_search_response(response)

<ipython-input-15-270633435>:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  response = client.search(



ID: zUkke5cBCq_DruP2sq4t
Score: 0.031513646
Title: Pulp Fiction
Runtime: 154
Plot: Pulp Fiction is a 1994 American independent crime film written and directed by Quentin Tarantino from a story he conceived with Roger Avary. It tells four intertwining tales of crime and violence in Los Angeles. The film stars John Travolta, Samuel L. Jackson, Bruce Willis, Tim Roth, Ving Rhames, and Uma Thurman. The title refers to the pulp magazines and hardboiled crime novels popular during the mid-20th century, known for their graphic violence and punchy dialogue.Tarantino wrote Pulp Fiction in 1992 and 1993, incorporating scenes that Avary originally wrote for True Romance (1993). Its plot occurs out of chronological order. The film is also self-referential from its opening moments, beginning with a title card that gives two dictionary definitions of pulp. Considerable screen time is devoted to monologues and casual conversations with eclectic dialogue revealing each character's perspectives on sev

In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-1.7B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# prepare the model input
prompt = "Give me a short introduction to large language model"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

tokenizer_config.json:   0%|          | 0.00/9.73k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/622M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

thinking content: <think>
Okay, the user wants a short introduction to large language models. Let me start by defining what they are. Large language models are AI systems trained on vast amounts of text data. I should mention their purpose, like generating text, understanding context, and adapting to different tasks.

Wait, I need to keep it concise. Maybe start with a simple definition. Then explain their training process—big data, deep learning. Highlight key features like natural language understanding and generation. Also, mention applications in various fields. But keep it short, so avoid too much technical jargon. Make sure it's easy to understand. Check if there's anything else important, like their limitations or future potential. But since it's a short intro, maybe just touch on that briefly. Alright, structure: definition, training, features, applications, and a sentence on future. That should cover it.
</think>
content: Large language models (LLMs) are advanced AI systems tr